In [1]:
import re
import requests
from bs4 import BeautifulSoup
import unidecode
import time
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime

**Extrai todos tickers disponíveis no site [fundamentus](https://fundamentus.com.br/detalhes.php)**

Utilizando a regex

```python
r'[A-Z]{4}\d{1,2}'
```

In [2]:
url_fundamentus = 'https://www.fundamentus.com.br/detalhes.php'
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'X-user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36 FKUA/website/41/website/Desktop',
    'Referer' :   'https://www.flipkart.com/sellers?pid=HSAFFJWT2WUYHHHX',
    'Content-Type': 'application/json',
    'Origin': 'https://www.flipkart.com',
    'Host': 'www.flipkart.com',
    'Pragma': 'no-cache'
}
rex_ticker = re.compile(r'[A-Z]{4}\d{1,2}')

In [3]:
response = requests.get(url_fundamentus, headers=headers)
TICKERS = [t.strip() for t in BeautifulSoup(response.text).find_all(string=rex_ticker)]

In [4]:
print(f'TOTAL DE TICKERS: {len(TICKERS)}')

TOTAL DE TICKERS: 938


**Itera sobre os tickers para pegar dados do balanco patrimonial e demonstrativos**

Algumas funções para tratar os textos do html:

In [5]:
def normalize(string: str):
    return unidecode.unidecode(string).upper().replace('.', '').replace(',', '').replace(' ', '_')

def clean_num(num: str):
    perc = '%' in num
    try:
        res = float(num.replace('.', '').replace(',', '.').replace('%', ''))
    except Exception as e:
        return float('nan')
    if perc:
        res /= 100
    return res

In [6]:
%%time
    
rows = []
for ticker in tqdm(TICKERS):
    try:

        row = dict(ticker=ticker)
        response = requests.get(url_fundamentus, params={'papel': ticker}, headers=headers)
        soup = BeautifulSoup(response.text)

        # BALANÇO PATRIMONIAL
        balanco = soup.find('span', 'txt', string='Dados Balanço Patrimonial').find_parent('table')
        data = zip(balanco.find_all('td', 'label'), balanco.find_all('td', 'data'))

        for k, v in data:
            key = normalize(k.find('span', 'txt').text)
            val = v.find('span', 'txt').text
            row[key] = clean_num(val)

        # DEMONSTRATIVOS
        demo = soup.find('span', 'txt', string='Dados demonstrativos de resultados').find_parent('table')
        data = zip(demo.find_all('td', 'label'), demo.find_all('td', 'data'))

        for k, v in data:
            key = normalize(k.find('span', 'txt').text)
            val = v.find('span', 'txt').text

            if key in row:
                continue
            else:
                 row[key] = clean_num(val)
        # OUTROS DADOS
        
        ## num de acoes
        n_acoes = soup.find('span', string='Nro. Ações')
        k = normalize(n_acoes.text)
        v = clean_num(n_acoes.parent.find_next_sibling('td', 'data').find('span', 'txt').text)
        row[k] = v
        
        ## crescimento receita (5 anos)
        cresc5a = soup.find('span', string='Cres. Rec (5a)')
        k = normalize(cresc5a.text)
        v = clean_num(cresc5a.parent.find_next_sibling('td', 'data').find('span', 'txt').text)
        row[k] = v

        ## ROIC
        roic = soup.find('span', string='ROIC')
        k = normalize(roic.text)
        v = clean_num(roic.parent.find_next_sibling('td', 'data').find('span', 'txt').text)
        row[k] = v
        
        ## Liquidez Corr
        liq_corr = soup.find('span', string='Liquidez Corr')
        k = normalize(liq_corr.text)
        v = clean_num(liq_corr.parent.find_next_sibling('td', 'data').find('span', 'txt').text)
        row[k] = v

        ## setor
        setor = soup.find('span', string='Setor')
        k = normalize(setor.text)
        v = setor.parent.find_next_sibling('td', 'data').find('span', 'txt').text
        row[k] = normalize(v)

        ## subsetor
        subsetor = soup.find('span', string='Subsetor')
        k = normalize(subsetor.text)
        v = subsetor.parent.find_next_sibling('td', 'data').find('span', 'txt').text
        row[k] = normalize(v)

        rows.append(row)
        time.sleep(1)
    except Exception as e:
        print(ticker, e)


ABRE3 'NoneType' object has no attribute 'find_parent'
AORE3 'NoneType' object has no attribute 'find_parent'
BPNM4 'NoneType' object has no attribute 'find_parent'
BRIN3 'NoneType' object has no attribute 'find_parent'
BVMF3 'NoneType' object has no attribute 'find_parent'
CELP5 'NoneType' object has no attribute 'find_parent'
CELP7 'NoneType' object has no attribute 'find_parent'
ESTC3 'NoneType' object has no attribute 'find_parent'
FJTA3 'NoneType' object has no attribute 'find_parent'
FJTA4 'NoneType' object has no attribute 'find_parent'
HRTP3 'NoneType' object has no attribute 'find_parent'
IDNT3 'NoneType' object has no attribute 'find_parent'
INET3 'NoneType' object has no attribute 'find_parent'
KROT3 'NoneType' object has no attribute 'find_parent'
MLFT3 'NoneType' object has no attribute 'find_parent'
PARC3 'NoneType' object has no attribute 'find_parent'
QGEP3 'NoneType' object has no attribute 'find_parent'
RNAR3 'NoneType' object has no attribute 'find_parent'
RUMO3 'Non

**Salvar num CSV datado**

In [7]:
df = pd.DataFrame(rows).set_index('ticker')

df.to_csv(f'../data/{datetime.now().strftime("%Y_%m_%d")}_fundamentus.csv')

df.head()

,ATIVO,DIV_BRUTA,DISPONIBILIDADES,DIV_LIQUIDA,ATIVO_CIRCULANTE,PATRIM_LIQ,RECEITA_LIQUIDA,EBIT,LUCRO_LIQUIDO,NRO_ACOES,CRES_REC_(5A),ROIC,LIQUIDEZ_CORR,SETOR,SUBSETOR,DEPOSITOS,CART_DE_CREDITO,RESULT_INT_FINANC,REC_SERVICOS
ticker,,,,,,,,,,,,,,,,,,,
AALR3,2.618910e+09,9.017330e+08,3.253200e+08,5.764130e+08,6.355580e+08,1.179760e+09,9.147390e+08,-2.432020e+06,-8.448000e+07,1.182930e+08,-0.040,-0.001,0.87,SAUDE,SERVICOS_MEDICO_-_HOSPITALARES_ANALISES_E_DIAG...,NaN,NaN,NaN,NaN
ABCB3,4.989580e+10,NaN,NaN,NaN,NaN,4.093300e+09,NaN,NaN,4.187330e+08,2.183590e+08,0.028,NaN,NaN,FINANCEIROS,BANCOS,7.325720e+09,1.768990e+10,-5453990.0,292984000.0
ABCB4,4.989580e+10,NaN,NaN,NaN,NaN,4.093300e+09,NaN,NaN,4.187330e+08,2.183590e+08,0.028,NaN,NaN,FINANCEIROS,BANCOS,7.325720e+09,1.768990e+10,-5453990.0,292984000.0
ABEV3,1.270570e+11,6.855010e+09,2.310340e+10,-1.624840e+10,3.909880e+10,7.800630e+10,5.567920e+10,1.378950e+10,8.693090e+09,1.573510e+10,0.045,0.155,1.29,BEBIDAS,CERVEJAS_E_REFRIGERANTES,NaN,NaN,NaN,NaN
ABYA3,9.782870e+08,3.831590e+08,2.955600e+07,3.536030e+08,4.982220e+08,2.920600e+08,2.506570e+08,2.580200e+07,-2.398000e+06,1.049060e+08,0.164,0.028,2.09,CONSTRUCAO_E_ENGENHARIA,CONSTRUCAO_CIVIL,NaN,NaN,NaN,NaN
